# Индекс аффинитивности по соц-дем
Пример расчета аффинитивности целевых аудиторий

## Описание задачи и условий расчета
- Период: 01.01.2022 - 31.01.2022
- Временной интервал: 05:00-29:00
- ЦА: Все 4+ в разбивке на целевые аудитории
- Место просмотра: Все места (дом+дача)
- Каналы: ТНТ и СТС									
- Статистики: TgSat%, TgAffin%, TgSatReach%, TgAffinReach%

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# В справочнике телесетей получим id нужных нам каналов
cats.get_tv_net(name=["ТНТ", "СТС"])

# Это id: 83, 11

In [ ]:
# В справочнике демографических атрибутов найдем необходимые нам переменные
cats.get_tv_demo_attribute(names=['пол','возраст','образование','род занятий','уровень дохода','питание'])

# Для формирования условий запроса нам понадобятся значения столбца entityName: sex, age, education, status, incLevel, spendingsOnFood

## Формирование задания
Зададим условия расчета

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. 
date_filter = [('2022-01-01', '2022-01-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы: ТНТ и СТС
company_filter = 'tvNetId IN (83, 11)'

# Указываем список статистик для расчета
statistics = ['TgSatPer','TgAffinPer','TgSatReachPer','TgAffinReachPer']

# Указываем срезы: телесеть
slices = ['tvNetName']

# Задаем условия сортировки
sortings = None

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

Сформируем словарь с целевыми аудиториями

In [ ]:
# Задаем необходимые группы
targets = {
    'Тотал':{'var_name': '4+', 'syntax': 'age >= 4', 'order':1},
    'Мужчины':{'var_name': 'Пол', 'syntax': 'sex = 1', 'order':2},
    'Женщины':{'var_name': 'Пол', 'syntax': 'sex = 2', 'order':3},
    '4-17':{'var_name': 'Возраст', 'syntax': 'age >= 4 AND age <= 17', 'order':4},
    '18-34':{'var_name': 'Возраст', 'syntax': 'age >= 18 AND age <= 34', 'order':5},
    '35-44':{'var_name': 'Возраст', 'syntax': 'age >= 35 AND age <= 44', 'order':6},
    '45-54':{'var_name': 'Возраст', 'syntax': 'age >= 45 AND age <= 54', 'order':7},
    '55+':{'var_name': 'Возраст', 'syntax': 'age >= 55', 'order':8},
    'Начальное, среднее':{'var_name': 'Образование', 'syntax': 'education IN (1, 2)', 'order':9},
    'Высшее':{'var_name': 'Образование', 'syntax': 'education = 3', 'order':10},
    'Руководитель':{'var_name': 'Род занятий', 'syntax': 'status IN (1,9,10)', 'order':11},
    'Рабочий':{'var_name': 'Род занятий', 'syntax': 'status = 4', 'order':12},
    'Дошкольник, студент, учащийся':{'var_name': 'Род занятий', 'syntax': 'status = 5', 'order':13},
    'Домохозяйка, молодая мать':{'var_name': 'Род занятий', 'syntax': 'status = 8', 'order':14},
    'Специалист, служащий':{'var_name': 'Род занятий', 'syntax': 'status IN (2, 3)', 'order':15},
    'Другое':{'var_name': 'Род занятий', 'syntax': 'status IN (6,7,11)', 'order':16},
    'Имеют возможность покупать только самое необходимое':{'var_name': 'Уровень дохода', 'syntax': 'incLevel IN (1,2)', 'order':17},
    'Хватает на еду и одежду, но не на дорогие вещи':{'var_name': 'Уровень дохода', 'syntax': 'incLevel = 3', 'order':18},
    'Имеют возможность покупать дорогие вещи':{'var_name': 'Уровень дохода', 'syntax': 'incLevel IN (4,5)', 'order':19},
    'Меньше четверти':{'var_name': 'Затраты на питание', 'syntax': 'spendingsOnFood = 1', 'order':20},
    'От четверти до половины':{'var_name': 'Затраты на питание', 'syntax': 'spendingsOnFood = 2', 'order':21},
    'От половины до трех четвертей':{'var_name': 'Затраты на питание', 'syntax': 'spendingsOnFood = 3', 'order':22},
    'Более трех четвертей':{'var_name': 'Затраты на питание', 'syntax': 'spendingsOnFood = 4', 'order':23},
    'Нет ответа':{'var_name': 'Затраты на питание', 'syntax': 'spendingsOnFood = 5', 'order':24}
}

## Расчет задания

In [ ]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой ЦА формируем задания и отправляем на расчет
for category, variable in targets.items():
    
    # Формируем задание на расчет ЦА
    project_name = f"{variable['order']}&{variable['var_name']}&{category}" 
    targetdemo_filter = f"{variable['syntax']}"
      
    # Формируем задание для API TV Index в формате JSON
    task_json = mtask.build_timeband_task(task_name=project_name, date_filter=date_filter, 
                                          weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                          company_filter=company_filter, time_filter=time_filter, 
                                          basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                          location_filter=location_filter, slices=slices, 
                                          statistics=statistics, sortings=sortings, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_timeband_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')
    
print(f"\nid: {[i['task']['taskId'] for i in tasks]}") 

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)

## Настройка внешнего вида таблицы

In [ ]:
# Разделим колонку с именем проекта на составные части: номер для сортировки, имя переменной и категорию
df[['Порядок','Переменная','Категория']] = df['prj_name'].str.split('&', 2, expand=True)

# Переведем формат колонки порядок в числовой для корректной сортировки
df = df.astype({'Порядок':int})

In [ ]:
# Формируем сводную таблицу: строки - ЦА, столбцы - каналы, значения - статистики
df = pd.pivot_table(df, values = statistics,
                        index=['Порядок','Переменная', 'Категория'], 
                        columns=['tvNetName'])

# Удаляем сортировочный столбец
df.index = df.index.droplevel(0)

df

In [ ]:
# Опционально: поменять местами уровни в столбцах
df = df.swaplevel(0,1, axis=1).sort_index(axis=1)

df

## Экспорт в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('03_affinity')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)